In [12]:
!pip install -U langchain-huggingface 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 固定示例

最基本（也是常见）的小样本提示技术是使用固定提示示例。这样您就可以选择一条链条，对其进行评估，并避免担心生产中的额外移动部件。

模板的基本组件是： 
- examples ：要包含在最终提示中的字典示例列表。 
- example_prompt ：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见的示例是将每个示例转换为一条人工消息和一条人工智能消息响应，或者一条人工消息后跟一条函数调用消息。

In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

In [2]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

组装成少示例的提示模板。

In [3]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [4]:
#最后，组装最终的提示并将其与模型一起使用。

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位非常厉害的数学天才。"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# openai_api_key="sk-9846f14a2104490b960adbf5c5b3b32e" #老师的Key
# openai_api_base = "https://api.deepseek.com"
# MODEL= "deepseek-chat"
# openai_api_key = "EMPTY"
# openai_api_base = "http://localhost:11434/v1"
# MODEL = "deepseek-r1:1.5b"
openai_api_key="sk-kEjwwIwe4w4EhkfGei4kNkawKIZeyEfezSdLfiyJrR4jdgPl"
openai_api_base = "https://api.fe8.cn/v1"
MODEL= "gpt-4o-mini"

chat = ChatOpenAI(
    model= MODEL,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.7,
)


output_parser = StrOutputParser()

chain = final_prompt | chat | output_parser

chain.invoke({"input":"3的平方是多少？"})

'3的平方是9。'

In [8]:
chat.invoke(input="3的平方是多少？")

AIMessage(content='3的平方是9。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 12, 'total_tokens': 19, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BRxGekD77DlkRmjMJ4ohvuxhA97cs', 'finish_reason': 'stop', 'logprobs': None}, id='run-2004c420-3bd1-4e51-86cf-69da5f6533d0-0', usage_metadata={'input_tokens': 12, 'output_tokens': 7, 'total_tokens': 19, 'input_token_details': {}, 'output_token_details': {}})

## 动态几次提示

有时您可能希望根据输入来限制显示哪些示例。为此，您可以将 examples 替换为 example_selector 。其他组件与上面相同！回顾一下，动态几次提示模板将如下所示：

- example_selector ：负责为给定输入选择少数样本（以及它们返回的顺序）。它们实现了 BaseExampleSelector 接口。一个常见的例子是向量存储支持的 SemanticSimilarityExampleSelector

- example_prompt ：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见的示例是将每个示例转换为一条人工消息和一条人工智能消息响应，或者一条人工消息后跟一条函数调用消息。

这些可以再次与其他消息和聊天模板组合以组合您的最终提示。

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_path = 'D:\\AInewModels\\BAAI\\bge-m3'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)


In [14]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "牛对月亮说了什么？", "output": "什么都没有"},
    {
        "input": "给我写一首关于月亮的五言诗",
        "output": "月儿挂枝头，清辉洒人间。 银盘如明镜，照亮夜归人。 思绪随风舞，共赏中秋圆。",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [15]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "对牛弹琴"})

[{'input': '牛对月亮说了什么？', 'output': '什么都没有'},
 {'input': '给我写一首关于月亮的五言诗',
  'output': '月儿挂枝头，清辉洒人间。 银盘如明镜，照亮夜归人。 思绪随风舞，共赏中秋圆。'}]

In [16]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [17]:
few_shot_prompt.format(input="What's 3+3?")

'Human: 2+3\nAI: 5\nHuman: 2+2\nAI: 4'

In [18]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位非常厉害的数学天才。"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [19]:
print(final_prompt.format(input="3+5是多少？"))

System: 你是一位非常厉害的数学天才。
Human: 2+3
AI: 5
Human: 2+2
AI: 4
Human: 3+5是多少？


In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
openai_api_key="sk-kEjwwIwe4w4EhkfGei4kNkawKIZeyEfezSdLfiyJrR4jdgPl"
openai_api_base = "https://api.fe8.cn/v1"
MODEL= "gpt-4o-mini"

chat = ChatOpenAI(
    model= MODEL,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.7,
)

output_parser = StrOutputParser()

chain = final_prompt | chat | output_parser

chain.invoke({"input":"3+5是多少？"})

'3 + 5 是 8。'